In [1]:
import json
import re

In [2]:
text = "(ACB) from 2020 to Q3 2024."

In [3]:
def check_pattern(text):
    # Define the regex pattern
    pattern = r'from \d{4} to Q[1-4] \d{4}'
    
    if re.search(pattern, text):
        return True
    else:
        return False

In [4]:
check_pattern(text)

True

In [18]:
data = dict()
questions = []
questions_ids = set()
with open('../data/generated_questions.json') as f:
    data2 = json.loads(f.read())
    for data_line in data2:
        line = data_line['ids']
        if data_line['version'] == 'v0':
            data[line] = dict()
            data[line]['year-to-quarter'] = data_line['year-to-quarter']
            if data_line['year-to-quarter']:
                questions.append(data_line)
                questions_ids.add(line)

len(data)

        

1481

In [19]:
len(questions)

528

In [20]:
with open('../data/generated_questions_2.json', 'w') as f:
    json.dump(questions, f, indent=4)

In [25]:
lines = []
with open('../data/deepseek-chat__v0.jsonl') as f:
    for line in f:
        data_line = json.loads(line)
        if data_line['ids'] not in questions_ids:
            lines.append(data_line)

In [23]:
lines

[{'ids': '3b7afc5b-54f8-4a82-9c8e-898202f7f3a0', 'score': 1},
 {'ids': '042a38a5-a2eb-4df3-ad29-9abe0e1798ac', 'score': 0},
 {'ids': '8d94dae4-e7c1-4c27-89d6-f77a2681c096', 'score': 1},
 {'ids': '774eddaf-1967-4ef3-82eb-3ba1cbc5f69b', 'score': 1},
 {'ids': 'd6230f84-3de5-441f-bc68-77b7d087a620', 'score': 0},
 {'ids': 'fc4ba458-07f7-47b6-acef-a58b4ce982f6', 'score': 1},
 {'ids': '78ea0d87-fe1b-4d65-8d48-4cf9cfb89307', 'score': 1},
 {'ids': '892af9dc-277f-41ab-912c-006545cca718', 'score': 1},
 {'ids': 'a1768a77-e435-47f2-9da6-c1ddbc8ab977', 'score': 1},
 {'ids': 'aaa3792b-b52a-4576-862d-a6a8d6da2623', 'score': 1},
 {'ids': '288c5f45-4768-4122-bea2-5e042eb3e527', 'score': 1},
 {'ids': 'f362d61f-4765-4b26-a6f8-01f632e1e2cb', 'score': 1},
 {'ids': '4172fecd-2b98-4a47-9230-eb5c4382ae31', 'score': 1},
 {'ids': '9a21690c-f1f1-48d7-81c6-63bc66ada36f', 'score': 0},
 {'ids': 'bb257498-2d4d-4484-9ee3-cacae00d9d81', 'score': 1},
 {'ids': '67644a21-6c8d-4f43-99cd-e23e5f6bb60b', 'score': 1},
 {'ids':

In [26]:
with open('../data/deepseek-chat__v0_2.jsonl', 'w') as f:
    for line in lines:
        json.dump(line, f)
        f.write('\n')

In [11]:
for line in lines:
    ids = line['ids']
    data[ids]['score'] = line['score']
    line['year-to-quarter'] = data[ids]['year-to-quarter']

In [12]:
lines[0]

{'ids': '3b7afc5b-54f8-4a82-9c8e-898202f7f3a0',
 'score': 1,
 'year-to-quarter': False}

In [9]:
score_0 = 0
y_q = 0
inner = 0

isolated = 0

for line in data:
    if data[line]['score'] == 0:
        score_0 += 1
    if data[line]['year-to-quarter']:
        y_q += 1
    if data[line]['score'] == 0 and data[line]['year-to-quarter']:
        inner += 1

In [10]:
score_0, y_q, inner

(657, 528, 325)

In [ ]:
isolated = []

In [27]:
import pandas as pd

df = pd.read_parquet('../data/industry_ratio_v3.parquet')
df

,industry,year,quarter,ratio_code,data_mean,date_added
0,Automobiles and Parts,2015,0,AFDAR,-0.003423,2015-12-30
1,Automobiles and Parts,2015,0,AFDATTAR,-0.000490,2015-12-30
2,Automobiles and Parts,2015,0,ATDR,1.321110,2015-12-30
3,Automobiles and Parts,2015,0,BEP,0.012869,2015-12-30
4,Automobiles and Parts,2015,0,BEPR,0.012869,2015-12-30
...,...,...,...,...,...,...
55897,"Utilities (Electricity, Water & Gas)",2024,3,STDTAR,0.362961,2024-09-30
55898,"Utilities (Electricity, Water & Gas)",2024,3,STDTER,0.974953,2024-09-30
55899,"Utilities (Electricity, Water & Gas)",2024,3,TAGYoY,0.139074,2024-09-30
55900,"Utilities (Electricity, Water & Gas)",2024,3,TAT,0.312422,2024-09-30
